In [1]:
import zipfile
import pandas as pd
import nltk

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lavri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
archive = zipfile.ZipFile('training_data.zip', 'r')
df = pd.read_csv(archive.open('training.1600000.processed.noemoticon.csv'), header=None, encoding_errors="replace")
# df = pd.read_csv("training_data/training.1600000.processed.noemoticon.csv", encoding="utf-8", encoding_errors="replace")

In [3]:
# target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

# ids: The id of the tweet ( 2087)

# date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

# flag: The query (lyx). If there is no query, then this value is NO_QUERY.

# user: the user that tweeted (robotickilldozr)

# text: the text of the tweet (Lyx is cool)

df.loc[df[0]==4, 0]= 1
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data = df.drop([1,2,3,4], axis=1)
# data.columns = ["label","text"]
data.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
# data = data["text"].dropna()
data[5] = [entry.lower() for entry in data[5]]
data.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [6]:
# data[5]= [word_tokenize(entry) for entry in data[5]]

In [7]:
# data.to_csv('tokenized_data.csv', index=False)
data = pd.read_csv('tokenized_data.csv')

In [8]:
data.head()

,0,5
0,0,"['@', 'switchfoot', 'http', ':', '//twitpic.co..."
1,0,"['is', 'upset', 'that', 'he', 'ca', ""n't"", 'up..."
2,0,"['@', 'kenichan', 'i', 'dived', 'many', 'times..."
3,0,"['my', 'whole', 'body', 'feels', 'itchy', 'and..."
4,0,"['@', 'nationwideclass', 'no', ',', 'it', ""'s""..."


In [9]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['5'],data['0'],test_size=0.20)

In [10]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['5'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [11]:
print(Tfidf_vect.vocabulary_)

{'http': 2153, 'twitpic': 4564, 'com': 945, 'awww': 400, 'that': 4342, 'bummer': 712, 'you': 4972, 'shoulda': 3862, 'got': 1877, 'david': 1162, 'of': 3066, 'third': 4373, 'day': 1165, 'to': 4431, 'do': 1293, 'it': 2283, 'is': 2277, 'upset': 4637, 'he': 2013, 'ca': 738, 'update': 4627, 'his': 2080, 'facebook': 1551, 'by': 736, 'texting': 4329, 'and': 248, 'might': 2795, 'cry': 1089, 'as': 330, 'result': 3602, 'school': 3736, 'today': 4433, 'also': 227, 'blah': 559, 'many': 2696, 'times': 4416, 'for': 1702, 'the': 4344, 'ball': 433, 'managed': 2687, 'save': 3721, '50': 86, 'rest': 3599, 'go': 1848, 'out': 3134, 'my': 2913, 'whole': 4807, 'body': 598, 'feels': 1610, 'itchy': 2286, 'like': 2550, 'its': 2290, 'on': 3091, 'fire': 1649, 'no': 3006, 'not': 3029, 'at': 350, 'all': 214, 'mad': 2663, 'why': 4815, 'am': 231, 'here': 2055, 'because': 482, 'see': 3770, 'over': 3140, 'there': 4357, 'crew': 1075, 'need': 2952, 'hug': 2155, 'hey': 2060, 'long': 2608, 'time': 4415, 'yes': 4959, 'rains':

In [12]:
print(Train_X_Tfidf)

  (0, 4853)	0.1933673490013036
  (0, 3006)	0.21603540828406553
  (0, 2853)	0.34608638461269225
  (0, 2402)	0.519546977799641
  (0, 2221)	0.16259089929785447
  (0, 2169)	0.36675975771570646
  (0, 617)	0.509546823769615
  (0, 350)	0.1956836912215655
  (0, 330)	0.2595271275122342
  (1, 3091)	1.0
  (2, 4577)	0.4092910028275572
  (2, 4459)	0.2846229428969209
  (2, 4431)	0.1517691782498754
  (2, 3029)	0.24985017584865343
  (2, 2614)	0.38989931507415576
  (2, 2380)	0.24011314671691805
  (2, 1717)	0.43178052038133435
  (2, 1165)	0.26758598305265596
  (2, 727)	0.44530796794379823
  (3, 4760)	0.22825669897181589
  (3, 4682)	0.49868973808458417
  (3, 4459)	0.22281217508967147
  (3, 4344)	0.12244829054000422
  (3, 3969)	0.1801127014843266
  (3, 2634)	0.22492258779737376
  :	:
  (1279998, 4836)	0.28753699278682165
  (1279998, 4431)	0.08463297115035145
  (1279998, 4250)	0.22945671791652864
  (1279998, 3548)	0.3456385435966356
  (1279998, 3066)	0.24370111588273952
  (1279998, 3029)	0.1393271214112716

In [13]:
# Naive Bayes Classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  76.9240625


In [14]:
# Support Vector Machine Classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)